# Notebook Data Preparation

This notebook shows how to extract data from wave files using extractor, and how the data is divided between test and training sets

In [203]:
from models import gmm, lstm
from extractor import extractor
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections import OrderedDict

In [204]:
importlib.reload(extractor)
wav_data = "/Users/oosv/data/voxforge" #Path to .wav file data
languages = {"english" : 0, "spanish" : 1, "french" : 2}
TOTAL_FRAMES = 150
ENERGY_MIN = 12
DELTA_DIM = 3
CEPSTRAL_COEF = 13

extractor.TOTAL_FRAMES = TOTAL_FRAMES # lenght of audio each clip will be divided into
extractor.MIN_ENERGY = ENERGY_MIN # Minimum threshold of energy, zero's out anything not within threshold

In [205]:
data_y = []
data_x = []
# num_samples = 100
# get the path for num_samples .wav files
for language in languages:
    
    files = []
    path = os.path.join(wav_data, language)
    for f in os.listdir(path):
        files.append(os.path.join(path, f))
    
    x, y = extractor.make_feature_set(files, languages[language])
    data_x.append(x)
    data_y.append(y)

/Users/oosv/miniconda3/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


In [206]:
print(len(data_y))
print(len(data_x))
data_x = np.concatenate(data_x, axis = 0)
data_y = np.concatenate(data_y, axis = 0)

3
3


In [207]:
shape_x, shape_y = np.shape(data_x), np.shape(data_y)
print(shape_x, shape_y)

(314809, 3, 150, 13) (314809,)


In [208]:
# Convert from Sup_Seq x 3 x TOTAL_FRAMES x n_coeff
#         to   Sup_Seq x TOTAL_FRAMES x n_coeff * 3
data_x = np.reshape(data_x, [shape_x[0], shape_x[2], shape_x[1] * shape_x[3]])
shape_x = np.shape(data_x)
print(shape_x)
print(f'total length of audio = {shape_x[0] * .150 / 3600} hours' )

(314809, 150, 39)
total length of audio = 13.117041666666665 hours


In [209]:
def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [210]:
# number of samples per language
counts = Counter(data_y)
print(counts)

Counter({0: 154673, 1: 96687, 2: 63449})


In [211]:
def make_equal(counts, data_x, data_y):
    low_idx = min(counts, key=counts.get)
    low = counts[low_idx]
    new_y, new_x = [],[]
    begin = 0
    for idx in range(len(counts)):
        new_y += data_y[begin: begin + low].tolist()
        new_x += data_x[begin: begin + low].tolist()
        begin += counts[idx]
    return new_x, new_y


    

In [ ]:
data_x, data_y = make_equal(counts, data_x, data_y)
print(np.shape(data_x), np.shape(data_y))

In [ ]:
shuffle_in_unison(data_x, data_y)
shape_x = np.shape(data_x)

In [217]:
print(shape_x)
print(f'total length of audio = {shape_x[0] * .150 / 3600} hours' )

(190347, 150, 39)
total length of audio = 7.931125 hours


In [218]:
train_x = data_x[int(shape_x[0]/4):]
test_x = data_x[:int(shape_x[0]/4)]
train_y = data_y[int(shape_x[0]/4):]
test_y = data_y[:int(shape_x[0]/4)]

In [219]:
print(np.shape(train_x), np.shape(train_y))
print(np.shape(test_x), np.shape(test_y))

(142761, 150, 39) (142761,)
(47586, 150, 39) (47586,)


In [220]:
with open('train_x_3.npy', 'wb') as file:
    np.save(file, train_x)
with open('train_y_3.npy', 'wb') as file:
    np.save(file, train_y)
with open('test_x_3.npy', 'wb') as file:
    np.save(file, test_x)
with open('test_y_3.npy', 'wb') as file:
    np.save(file, test_y)